In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import tensorflow_datasets as tfds
import pathlib
import os

# 1- input pipeline using tf.data

In [ ]:
#training_data_path = pathlib.Path(r"H:\AI\DataSets\275 Bird Species also see 73 Sports Dataset\birds_rev2\train")
#validatoin_data_path = pathlib.Path(r"H:\AI\DataSets\275 Bird Species also see 73 Sports Dataset\birds_rev2\valid")
#testing_data_path = pathlib.Path(r"H:\AI\DataSets\275 Bird Species also see 73 Sports Dataset\birds_rev2\test")

#batch_size = 64
#img_height = 224
#img_width = 224

#train_image_count = len(list(training_data_path.glob(r'*\\*.jpg')))

#train_list_ds = tf.data.Dataset.list_files([str(training_data_path/'*/*'), 
#                                           str(validatoin_data_path/'*/*'),
#                                           str(testing_data_path/'*/*')], shuffle=False)
#train_list_ds = train_list_ds.shuffle(train_image_count, reshuffle_each_iteration=False)
#------------------------------------------------------------------------------------------------------
#val_image_count = len(list(validatoin_data_path.glob(r'*\\*.jpg')))

#val_list_ds = tf.data.Dataset.list_files(str(validatoin_data_path/'*/*'), shuffle=False)
#val_list_ds = val_list_ds.shuffle(val_image_count, reshuffle_each_iteration=False)
#------------------------------------------------------------------------------------------------------
#test_image_count = len(list(training_data_path.glob(r'*\\*.jpg')))

#test_list_ds = tf.data.Dataset.list_files(str(testing_data_path/'*/*'), shuffle=False)
#test_list_ds = test_list_ds.shuffle(test_image_count, reshuffle_each_iteration=False)

In [ ]:
training_data_path = pathlib.Path('../input/100-bird-species/birds_rev2/train')
validatoin_data_path = pathlib.Path('../input/100-bird-species/birds_rev2/valid')
testing_data_path = pathlib.Path('../input/100-bird-species/birds_rev2/test')

batch_size = 64
img_height = 150
img_width = 150


image_count = len(list(training_data_path.glob(r'*/*.jpg')))

all_list_ds = tf.data.Dataset.list_files([str(training_data_path/'*/*'), 
                                          str(validatoin_data_path/'*/*'),
                                          str(testing_data_path/'*/*')], shuffle=False)
all_list_ds = all_list_ds.shuffle(image_count, reshuffle_each_iteration=False)


val_size = int(image_count * 0.2)
#reduce the training set size fue to the small memory allocated for this notebook
skip_train = int(image_count * 0.6)
train_ds = all_list_ds.skip(skip_train)
val_ds = all_list_ds.take(val_size)

In [ ]:
class_names = np.array(sorted([item.name for item in training_data_path.glob('*')]))
num_classes = len([item for item in training_data_path.glob('*')])
print("The number of classes is: {}".format(num_classes))

In [ ]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

**Write a short function that converts a file path to an (img, label) pair:**

For more details:

https://www.tensorflow.org/tutorials/load_data/images#using_tfdata_for_finer_control

In [ ]:
def process_path(file_path):
    
    parts = tf.strings.split(file_path, os.path.sep)
    one_hot = parts[-2] == class_names
    label = tf.argmax(one_hot)
    
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])

    return img, label

**Use `Dataset.map` to create a dataset of image, label pairs.**

For more details:

https://www.tensorflow.org/tutorials/load_data/images#using_tfdata_for_finer_control

Set `num_parallel_calls` so multiple images are loaded/processed in parallel.

In [ ]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
#dead code

#train_ds = train_list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
#val_ds = val_list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
#test_ds = test_list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

**Configure dataset for performance** <br>

To train a model with this dataset you will want the data:<br>

- To be well shuffled.<br>
- To be batched.<br>
- Batches to be available as soon as possible.<br>

These features can be added using the tf.data API. For more details:

https://www.tensorflow.org/tutorials/load_data/images#using_tfdata_for_finer_control


`.cache()` keeps the images in memory after they're loaded off disk during the first epoch.
This will ensure the dataset does not become a bottleneck while training your model.
If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

`.prefetch()` overlaps data preprocessing and model execution while training.

source: https://www.tensorflow.org/tutorials/load_data/images#configure_the_dataset_for_perfor

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1024)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

#dead code
#train_ds = configure_for_performance(train_ds)
#val_ds = configure_for_performance(val_ds)
#test_ds = configure_for_performance(test_ds)

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

**Visualize the data**

For more details:

https://www.tensorflow.org/tutorials/load_data/images#using_tfdata_for_finer_control

In [ ]:
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    label = label_batch[i]
    plt.title(class_names[label])
    plt.axis("off")

# 2- Data augmentation Using tf.image

A technique to increase the diversity of your training set by applying random (but realistic) transformations such as image rotation

There are a variety of preprocessing layers you can use for data augmentation including RandomContrast, RandomCrop, RandomZoom, RandomFlip, RandomRotation and others.

Tere are two ways to implement data augmentation usin tensorflow. the first approch is Make the preprocessing layers part of the model, here When you export your model using model.save, the preprocessing layers will be saved along with the rest of your model. this will help us when we deploy the model it will automatically standardize images.The second approch is Apply the preprocessing layers to your dataset

source: https://www.tensorflow.org/tutorials/images/data_augmentation

I will use the second option in this notebook.

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", seed=0),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2, seed=0),
    tf.keras.layers.experimental.preprocessing.RandomCrop(height=150, width=150, seed=0),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.3, width_factor=0.3, seed=0)])

In [ ]:
#train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
#                        num_parallel_calls=tf.data.AUTOTUNE)

# 3- creating a simple model as a baseline

**define the convolutional base using a common pattern: a stack of `Conv2D` and `MaxPooling2D` layers.**

The CNN takes a tensor of shape (height, width, channels) 3D tensor (R, G, B) without givving it the batch size.
W will give the input shape to your first layer (180, 180, 3) by passing the argument `input_shape`.

source: https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
baseline_model = tf.keras.models.Sequential()

baseline_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(150, 150, 3) ))
baseline_model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

baseline_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
baseline_model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

baseline_model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
baseline_model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

baseline_model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
baseline_model.add(tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu'))
baseline_model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3)))

In [ ]:
baseline_model.summary()

The output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). as we can see
the height and the width shrink as we go depper in the network, we can control the number of output channels 
by for each Conv2D by the first argument (e.g., filters=32)

source: https://www.tensorflow.org/tutorials/images/cnn

**Add Dense layers on top**

Finally we will feed the last output tensor from the the convolutional base (of shape (20, 20, 64)) into Dense layers
that will perform the classification. First we need to Flatten the 3D tensor to 1D a `Flatten` layer, then we will add 
one or more dense layers on top of the flatten layer, we have 275 classes so the final dense layer 
will have 275 units.


source: https://www.tensorflow.org/tutorials/images/cnn


The final layer I will use the softmax activations that used for multicalss classification, 
for the  layer that preceding the final layer I will use 512 units to avoid information bottleneck.


After the first two layers I will add `BatchNormalization` layer to reduce the danger of **Vanishing/Exploding Gradient** 
problems 

In [ ]:
baseline_model.add(tf.keras.layers.Flatten())

#dead code
#baseline_model.add(tf.keras.layers.Dense(1024, activation='relu',
#                                        kernel_initializer='he_normal'))
#baseline_model.add(tf.keras.layers.BatchNormalization())
#baseline_model.add(tf.keras.layers.Dropout(0.4))

baseline_model.add(tf.keras.layers.Dense(512, activation='relu',
                                        kernel_initializer='he_normal'))
baseline_model.add(tf.keras.layers.BatchNormalization())
baseline_model.add(tf.keras.layers.Dropout(0.4))

baseline_model.add(tf.keras.layers.Dense(275, activation='softmax'))

In [ ]:
baseline_model.summary()

**Compile and train the model**

In [ ]:
baseline_model.compile(optimizer='adam',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                       metrics=['accuracy'])

history = baseline_model.fit(train_ds, epochs=15, 
                             validation_data=val_ds)

**Plotting the training and the validation losses and accuracies as a function of the number of iterations**

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']


plt.plot(history.epoch, train_loss, label='Training Loss')
plt.plot(history.epoch, val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(history.epoch, train_acc, label='Training Accuracy')
plt.plot(history.epoch, val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
# dead code

#testset_loss, testest_acc = baseline_model.evaluate(test_ds)
#print("The test set loss: {}, Test set Accuracy: {}".format(testset_loss, testest_acc))

# 4- Simple Inception Network

In [ ]:
def InceptionBlock(previous_output):
    
    kernel_init = tf.keras.initializers.glorot_uniform()
    
    conv_1x1_1 = tf.keras.layers.Conv2D(32, (1, 1), padding='same', activation='relu',
                                        kernel_initializer=kernel_init)(previous_output)
    
    conv_3x3 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu',
                                      kernel_initializer=kernel_init)(previous_output)
    conv_1x1_2 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu',
                                        kernel_initializer=kernel_init)(conv_3x3)
    
    conv_5x5 = tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu',
                                      kernel_initializer=kernel_init)(previous_output)
    conv_1x1_3 = tf.keras.layers.Conv2D(16, (1, 1), padding='same', activation='relu',
                                        kernel_initializer=kernel_init)(conv_5x5)
    
    conv_1x1_4 = tf.keras.layers.Conv2D(32, (1, 1), padding='same', activation='relu',
                                        kernel_initializer=kernel_init)(previous_output)
    maxpool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='same')(conv_1x1_4)
    
    concat = tf.keras.layers.concatenate([conv_1x1_1, conv_1x1_2, conv_1x1_3, maxpool], axis=-1)
    
    return concat

**Adding an Auxiliary output for fitting gradient vanishing and for regularization**

In [ ]:
def aux_output(previous_output):
    
    
    conv1 = tf.keras.layers.Conv2D(128, kernel_size=(1, 1), padding='same',
                                     activation='relu')(previous_output)
    
    avg_pooling = tf.keras.layers.AveragePooling2D(5, strides=3)(conv1)
    
    x = tf.keras.layers.Flatten()(avg_pooling)
        
    x = tf.keras.layers.Dense(512, activation='relu',
                              kernel_initializer='he_normal')(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    aux_out = tf.keras.layers.Dense(275, activation='softmax', name='auxiliary_output')(x)
    
    return aux_out

In [ ]:
model_input = tf.keras.Input(shape=(150, 150, 3))

x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), activation='relu')(model_input)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1))(x)

x = InceptionBlock(x)
x = InceptionBlock(x)
aux_out_1 = aux_output(x)
x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

x = InceptionBlock(x)
x = InceptionBlock(x)

x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=5)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu',
                                        kernel_initializer='he_normal')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
model_output = tf.keras.layers.Dense(275, activation='softmax', name='main_output')(x)

inc_model = tf.keras.models.Model(inputs=model_input, outputs=[model_output, aux_out_1])


In [ ]:
tf.keras.utils.plot_model(inc_model,
                          show_shapes=True,
                          show_dtype=True,
                          show_layer_names=True)

In [ ]:
inc_model.summary()

In [ ]:
inc_model.compile(optimizer='adam',
                  loss=[tf.keras.losses.SparseCategoricalCrossentropy(),
                       tf.keras.losses.SparseCategoricalCrossentropy()],
                  metrics=['accuracy'])



history = inc_model.fit(train_ds, epochs=25, 
                        validation_data=val_ds)

In [ ]:
train_loss = history.history['main_output_loss']
val_loss = history.history['val_main_output_loss']


plt.plot(history.epoch, train_loss, label='Training Loss')
plt.plot(history.epoch, val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
train_acc = history.history['main_output_accuracy']
val_acc = history.history['val_main_output_accuracy']

plt.plot(history.epoch, train_acc, label='Training Accuracy')
plt.plot(history.epoch, val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()